In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
URL = "https://www.macrotrends.net/stocks/charts/AAPL/apple/financial-ratios?freq=Q"
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
pattern = re.compile(r"var originalData = \[(.*?)\]", re.MULTILINE | re.DOTALL)

In [5]:
scriptParse = re.search(pattern, str(soup))

In [6]:
print(scriptParse.groups()[0])

{"field_name":"<a href='\/stocks\/charts\/AAPL\/apple\/current-ratio'>Current Ratio<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'AAPL', s: 'current-ratio', freq: 'Q', statement: 'financial-ratios' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2022-09-30":"0.87940","2022-06-30":"0.86460","2022-03-31":"0.92680","2021-12-31":"1.03780","2021-09-30":"1.07460","2021-06-30":"1.06190","2021-03-31":"1.14170","2020-12-31":"1.16300","2020-09-30":"1.36360","2020-06-30":"1.46950","2020-03-31":"1.49600","2019-12-31":"1.59780","2019-09-30":"1.54010","2019-06-30":"1.50460","2019-03-31":"1.31540","2018-12-31":"1.30060","2018-09-30":"1.13290","2018-06-30":"1.30730","2018-03-31":"1.45600","2017-12-31":"1.24200","2017-09-30":"1.27610","2017-06-30":"1.38830","2017-03-31":"1.39060","2016-12-31":"1.22820","2016-09-30":"1.35270","2016-06-30":"1.31160","2016-03-31":"1.28310","2015-12-31":"1.00170","2015-09-30":"1.10880","2

In [7]:
import json

In [8]:
block = scriptParse.groups()[0]

In [9]:
res = json.loads("".join(["[", block, "]"]))

In [10]:
res

[{'field_name': "<a href='/stocks/charts/AAPL/apple/current-ratio'>Current Ratio</a>",
  'popup_icon': '<div class=\'ajax-chart\' data-tipped-options="ajax: {data: { t: \'AAPL\', s: \'current-ratio\', freq: \'Q\', statement: \'financial-ratios\' }}"><i style=\'font-size:18px; color:#337ab7;\' class=\'fas fa-chart-bar\'></i></span></div>',
  '2022-09-30': '0.87940',
  '2022-06-30': '0.86460',
  '2022-03-31': '0.92680',
  '2021-12-31': '1.03780',
  '2021-09-30': '1.07460',
  '2021-06-30': '1.06190',
  '2021-03-31': '1.14170',
  '2020-12-31': '1.16300',
  '2020-09-30': '1.36360',
  '2020-06-30': '1.46950',
  '2020-03-31': '1.49600',
  '2019-12-31': '1.59780',
  '2019-09-30': '1.54010',
  '2019-06-30': '1.50460',
  '2019-03-31': '1.31540',
  '2018-12-31': '1.30060',
  '2018-09-30': '1.13290',
  '2018-06-30': '1.30730',
  '2018-03-31': '1.45600',
  '2017-12-31': '1.24200',
  '2017-09-30': '1.27610',
  '2017-06-30': '1.38830',
  '2017-03-31': '1.39060',
  '2016-12-31': '1.22820',
  '2016-09-

In [11]:
from datetime import datetime
format_yyyymmdd = "%Y-%m-%d"

In [12]:
financial_df = pd.DataFrame()

for financial in res:
    temporal_df = pd.DataFrame()
    dict_parsed = dict()
    
    # Take the name of the financial metric
    link = BeautifulSoup(str(financial['field_name']), "html.parser")
    name = link.get_text()
    
    # Take the values of these financial metric
    for key, value in zip(financial.keys(), financial.items()):
        try: 
            date = datetime.strptime(key, format_yyyymmdd)
            dict_parsed[value[0]] = value[1]

        except:
            continue
            
    temporal_df = pd.DataFrame.from_dict(dict_parsed, orient='index', columns=[name])
    financial_df = financial_df.join(temporal_df, how='outer')


In [13]:
financial_df

,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2022-09-30,0.87940,0.66140,2.36950,42.25920,27.61520,27.61520,,27.35230,22.98610,0.25550,10.52390,1.47950,60.83330,40.89240,40.89240,5.87400,13.84810,3.17820,1.4544,1.2547
2022-06-30,0.86460,0.61970,2.05980,43.25630,27.81620,27.81620,,27.80410,23.43570,0.24670,8.66450,1.96390,45.82720,33.45900,33.45900,5.78100,12.72320,3.61020,1.4474,1.3153
2022-03-31,0.92680,0.60520,1.78020,43.74990,30.81790,30.81790,,30.98230,25.70980,0.27740,10.02180,2.14270,42.00330,37.10740,37.10740,7.13220,14.64960,4.15850,1.7372,1.5828
2021-12-31,1.03780,0.59720,1.70710,43.76380,33.47290,33.47290,,33.27360,27.93980,0.32520,11.86220,1.89950,47.38210,48.14270,48.14270,9.08470,19.39390,4.40200,2.84310,2.67340
2021-09-30,1.07460,0.63360,1.97680,42.19530,28.53410,28.53410,,27.88870,24.65330,0.23750,7.32310,1.61850,55.60870,32.57410,32.57410,5.85500,11.93470,3.84070,1.1731,0.9843
2021-06-30,1.06190,0.62200,1.89470,43.29270,29.62650,29.62650,,29.92490,26.70140,0.24690,8.91830,2.40160,37.47480,33.82700,33.82700,6.59230,12.78820,3.88240,1.2895,1.1618
2021-03-31,1.14170,0.61100,1.75840,42.50650,30.70080,30.70080,,31.26790,26.37750,0.26570,9.86870,2.71170,33.18940,34.15830,34.15830,7.00860,13.28870,4.14580,1.4413,1.305
2020-12-31,1.16300,0.59990,1.69190,39.77780,30.09180,30.09180,,30.13220,25.80340,0.31480,13.49510,1.90100,47.34250,43.42080,43.42080,8.12160,17.37410,3.93650,2.26500,2.06050
2020-09-30,1.36360,0.60160,1.72080,38.16040,22.83690,22.83690,,23.03160,19.58790,0.19980,9.85200,1.72780,52.08890,19.39580,19.39580,3.91280,7.72720,3.84870,1.1524,1.0526
2020-06-30,1.46950,0.56540,1.55950,37.99950,21.93350,21.93350,,22.01060,18.85400,0.18810,9.30240,1.86080,48.36640,15.56820,15.56820,3.54600,6.76550,4.21820,0.9626,0.8701
